<a href="https://colab.research.google.com/github/RMeuter/NLPHumourProjet/blob/ironie/ironie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and dependencies

## Téléchargement / Imports des librarys et des fichiers

Installation :


In [ ]:
!pip3 install langdetect
!pip3 install stanza
!pip3 install nltk
!pip3 install pygrammalecte

Imports :


In [18]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
import stanza
from stanza.pipeline.processor import Processor, register_processor
from langdetect import *
#from custom_process import *
from collections import Counter
import nltk

from pathlib import Path
from pygrammalecte import*
# ATTENTION necessite le fichier mots_fr.py (dico mots français) et dico.py (dico verbes et conjugaions français)

from google.colab import drive
drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/Colab Notebooks/SMS/"
# PATH = "content/drive/MyDrive/Colab Notebooks/SMS/"
sys.path.append(PATH)
import correcteur as cor
# ATTENTION necessite le fichier corrector.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load / Download Stanzas and NLTK dependencies

In [19]:
stanza.download("fr") 
nltk.download("stopwords")

2021-04-29 15:03:40 INFO: Downloading default packages for language: fr (French)...
2021-04-29 15:03:42 INFO: File exists: /root/stanza_resources/fr/default.zip.
2021-04-29 15:03:50 INFO: Finished downloading models and saved to /root/stanza_resources.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Corpus Traitment

## Notre corpus de sms / Source : 88milSMS_88522_formated.csv

Dataframe :

In [20]:
FILENAME = "/content/drive/MyDrive/Colab Notebooks/SMS/SMS/88milSMS_88522_formated.csv" # "data/88milSMS_88522_formated.csv"

dfSms = pd.read_csv(FILENAME, sep=',')
# dfSms = dfSms.rename(columns={dfSms.columns[0] : "Index"}).set_index(dfSms.columns[0])
# dfSms = dfSms.set_index(dfSms.cols[0])

dfSms.head()

,Unnamed: 0,NUM_SMS,DATETIME,ID_NUM_TEL,sms,langue
0,0,1,15 sept. 2011 07:28:55,477,Hey ca va?\nAlors cette rentree?\nVa falloir s...,fr
1,1,2,15 sept. 2011 08:02:08,477,Ok super merci! Oui j'y comprends rien du tout...,fr
2,2,3,15 sept. 2011 08:03:01,477,Coucou !\nC'est quand la feria de Nimes? \nJ's...,fr
3,3,4,15 sept. 2011 08:03:35,477,Coucou :)\nOui ca c'est bien passe!\nAlors je ...,fr
4,4,5,15 sept. 2011 09:05:28,477,On peut se rejoindre quelque part? Tu as cours...,fr


## Dico python contenant nos balises en clé et leurs remplaçants


In [21]:
listePRE = {
    "PRE_" : "prénom",
    "SUR_" : "surnom",
    "TEL_" : "téléphone",
    "LIE_" : "lieu",
    "NOM_" : "nom",
}

listePRE

{'LIE_': 'lieu',
 'NOM_': 'nom',
 'PRE_': 'prénom',
 'SUR_': 'surnom',
 'TEL_': 'téléphone'}

## Dico émotions FEEL / Source : FEEL.csv

In [22]:
# Liste des colonnes de Feel qui nous interesse pour notre calcul de le poids des émotions d'un mot
feelings_col = ["joy", "fear", "sadness", "anger", "surprise", "disgust"]
dFeel = pd.read_csv(PATH + "SMS/FEEL.csv", sep=';')

# Ajout d'une colonne 'poids' dans la matrice dFeel contenant le poids des émotions d'un mot
dFeel['poids'] = dFeel.apply(lambda x : sum(x[feelings_col]) * -1 if x['polarity'] == "negative" else 1, axis = 1)

dFeel.head()

,id,word,polarity,joy,fear,sadness,anger,surprise,disgust,poids
0,1,à ce endroit là,positive,0,0,0,0,0,0,1
1,2,à le hâte,negative,0,1,0,0,1,0,-2
2,3,à part,negative,0,0,1,0,0,0,-1
3,4,à pic,negative,0,1,0,0,0,0,-1
4,5,à rallonge,negative,0,0,1,0,0,0,-1


# PIPELINE STANZA

## Attribution de poids associé à l'émotion que le mots possède


In [23]:
# Fonction emotion qui retourne un dictionnaire contenant le poids de chaque mot d'un texte cible
def emotion(text):
        sentiment = {}
        for w in text.split(" "):                    
            for i in range(dFeel.shape[0]):
                elem = dFeel.iloc[i, :]
                if w.find(elem.word) != -1:
                    sentiment[w]= elem.poids
        return sentiment

# Processeur feel qui ajoute une propriétés feel_weight qui attribut à chaque mot un poids d'émotion (qui est la somme des différentes colonnes émotions de notre dFeel)
@register_processor("feel")
class LowercaseProcessor(Processor):
    _requires = set(["tokenize","mwt"])
    _provides = set(["feel"])

    def __init__(self, config, pipeline, use_gpu):
        pass

    def _set_up_model(self, *args):
        pass
    
    def process(self, doc):
        feel_fr={}
        for i,sent in enumerate(doc.sentences):
            feel_fr[i]=emotion(sent.text)
        doc.add_property('feel_weight', default={}, getter=lambda self: feel_fr, setter=None)
        return doc
    

## Correction orthographique et grammaticale


### Correction orthographique

In [8]:
# Processeur corrector qui agit comme un correcteur orthographique (1er partie de notre correcteur automatique)
# Arborescence et distance Leveinstein
@register_processor("corrector")
class EmotionsProcessor(Processor):
    _requires = set(["tokenize", "lowercase", "stopword"])
    _provides = set(["corrector"])

    def __init__(self, config, pipeline, use_gpu):
        pass

    def _set_up_model(self, *args):
        pass

    def _filter_prop(self, err, props):
        res = {
            "max" : [],
            "normal" : [],
            "min" : []
        }
        for item in props:
            # print("filter_prop({},{}) : {}".format(err,props,item))
            if len(err) == len(item):
                res["max"].append(item)
            else:
                res["normal"].append(item)
        return res

    def _correct_aux(self, sent):
        res = ""
        for w in sent.words:
            erreurs = cor.verification(sent.text)
            for err in erreurs:
                prop = cor.propositions(cor.arbre, err, 1)
                prop = self._filter_prop(err, prop)
                # print("[{}] : {}".format(err, prop))
                if prop["max"] != []:
                    w.text = w.text.replace(err, prop["max"][0])
                elif prop["normal"] != []:
                    w.text = w.text.replace(err,prop["normal"][0])
        res += "{}\n".format(sent.text)
            # i += 1
        return res

    def process(self, doc):
        for sent in doc.sentences:
            sent = self._correct_aux(sent)

        return doc
    

### Correction grammaticale

## Filtre des identifiants

Certains sms comportait à l'origine un prénom, un numéro de téléphone ou bien un adresse. Ces informations ont été cachées en les identifiants de la manière suivantes : $<id>$ .

In [25]:
# Processeur identifier qui remplace chaque balise <X_> par son remplaçant
# ATTENTION : lors du réedit de listePRE il faut penser à le modifier dans _deletePre()
@register_processor("identifier")
class LowercaseProcessor(Processor):
    _requires = set(["tokenize","mwt"])
    _provides = set(["identifier"])

    def __init__(self, config, pipeline, use_gpu):
        pass

    def _set_up_model(self, *args):
        pass
    
    def _deletePre(self, sent):
        listePRE={
            'PRE_':"prénom",
            'SUR_':"surnom",
            'TEL_':"téléphone",
            'LIE_':"lieu",
            'NOM_':"nom"
        }
        #doc = nlp(sent)
        for w in sent.words:
            for pre in listePRE.keys():
                if w.text.find(pre)!=-1:
                    w.text=listePRE[pre]
        return sent
                    
                
    def process(self, doc):
        for sent in doc.sentences:
            sent.text=self._deletePre(sent)
        return doc

## Passe les chaînes de caractères en lowercase


In [26]:
#Processeur lowercase qui mets en minuscule chacun de nos tokens.
@register_processor("lowercase")
class LowercaseProcessor(Processor):
    _requires = set(["tokenize","mwt","identifier"])
    _provides = set(["lowercase"])

    def __init__(self, config, pipeline, use_gpu):
        pass

    def _set_up_model(self, *args):
        pass

    def process(self, doc):
        doc.text = doc.text.lower()
        for sent in doc.sentences:
            for tok in sent.tokens:
                tok.text = tok.text.lower()

            for word in sent.words:
                word.text = word.text.lower()

        return doc

## Retire les mots inutiles ou ne possèdant pas de poids émotionnel

In [30]:
# Processeur stopword qui enleve les stopword (en utilisant le dictionnaire des stopwords) nltk
# Et qui enleve la ponctuation de nos tokens (ascii de 33 à 64)
@register_processor("stopword")
class StopwordProcessor(Processor):
    ''' Processor that removes all "useless" words '''
    _requires = set(["tokenize","mwt","lowercase","identifier"])
    _provides = set(["stopword"])

    def __init__(self, config, pipeline, use_gpu):
        nltk.download("stopwords")
        self.stopwords = nltk.corpus.stopwords.words("french")
        self.punctuation = [str(chr(i)) for i in range(33, 64)] # ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-']

    def _set_up_model(self, *args):
        pass

    def _stopword_aux(self, sent):
        to_del = []
        for i, w in enumerate(sent.words):
            if w.text in self.stopwords or w.text in self.punctuation: # stopwords from a nltk dictionnary
                to_del.append(i)
            else:
                for c in w.text:
                    if c in self.punctuation: # ascii between 33 and 45
                        # print("in", c)
                        w.text = w.text.replace(c, ' ')

        for i in to_del[::-1]: # cross the list in reversed order to avoid out of range problems
            del sent.tokens[i]

        print("_stopword_aux : {}".format(sent))
        return sent

    def process(self, doc):
        for sent in doc.sentences:
            sent = self._stopword_aux(sent)

        return doc

## Stanza NLP Pipeline

Notre objectif :

1.   Correction automatique
2.   Élément de liste



In [31]:
# NLP init pipeline
nlp = stanza.Pipeline(lang='fr', processors='tokenize,mwt,identifier,stopword,lowercase,corrector,feel,lemma')
doc = nlp("Le <SUR_4> te souhaite aussi un tres joyeux anniv' !")

ValueError: ignored

# Results

In [32]:
# doc

print(doc.feel_weight)

{0: {'souhaite': 1, 'joyeux': 1, "anniv'": 1}}


________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

Ajout des cents mots les plus fréquent dans le corpus sms dans le dico stopwords

In [33]:
sms10=dfSms.sms.iloc[:10]
sms10

0    Hey ca va?\nAlors cette rentree?\nVa falloir s...
1    Ok super merci! Oui j'y comprends rien du tout...
2    Coucou !\nC'est quand la feria de Nimes? \nJ's...
3    Coucou :)\nOui ca c'est bien passe!\nAlors je ...
4    On peut se rejoindre quelque part? Tu as cours...
5    Encore moi... J'te harcele, oui oui et j'assum...
6    Dis, pour le premier midi on s'fait a manger d...
7    Hey hey !\nJ'te souhaite un tres joyeux n'anni...
8    Je suis desolee... J'avais pas vu l'heure et p...
9    Ah d'accord.\nCa me fait vraiment plaisir. Mer...
Name: sms, dtype: object

In [34]:
# nlp = stanza.Pipeline(lang='fr', processors='tokenize')
listeFrequence = dict()
for sms in sms10:
    doc = nlp(sms)
    for sentence in doc.sentences:
        for token in sentence.tokens:
            if token.text in listeFrequence.keys():
                listeFrequence[token.text]+=1
            else:
                listeFrequence[token.text]=1
            
print(listeFrequence)

{'Hey': 2, 'ca': 2, 'va': 1, '?': 9, 'Alors': 2, 'cette': 2, 'rentree': 1, 'Va': 1, 'falloir': 1, 'se': 3, 'trouver': 1, 'un': 3, "p'tit": 1, 'creneau': 1, 'pour': 4, 'voir': 1, '!': 15, 'Dis': 2, 'moi': 6, ',': 6, 'est': 3, '-ce': 1, 'que': 2, 'tu': 1, 'sais': 2, 'quand': 2, 'commence': 2, 'les': 2, 'cours': 3, 'de': 10, 'langue': 1, 'non': 1, 'specialiste': 1, 'Bisoux': 4, 'Ok': 1, 'super': 1, 'merci': 1, 'Oui': 2, "j'": 5, 'y': 2, 'comprends': 1, 'rien': 1, 'du': 2, 'tout': 2, 'a': 7, 'fac': 1, ':p': 1, "J'": 6, 'irai': 1, 'me': 2, 'renseigner': 1, 'aussi': 2, '(': 1, 'si': 1, 'trouve': 1, 'le': 3, 'batiment': 1, ':': 2, ')': 3, 'Merci': 2, 'encore': 1, 'Bonne': 1, 'soiree': 1, 'bisoux': 1, 'Coucou': 2, "C'": 1, 'la': 2, 'feria': 1, 'Nimes': 1, 'suis': 3, 'pas': 8, 'bien': 3, 'sure': 2, 'faire': 1, 'mais': 2, 'pourquoi': 1, 'Sinon': 1, 'oui': 3, 'profite': 1, 'ma': 1, 'derniere': 1, 'semaine': 1, 'vacance': 1, 'Bon': 1, 'courage': 1, ':)': 1, "c'": 1, 'passe': 1, 'je': 4, 'jeudi': 1

In [16]:
# Premièrement, on récupère la fréquence totale de chaque mot sur tout le corpus d'artistes
freq_totale = nltk.Counter()
for k, v in corpora.iteritems():
    freq_totale += freq[k]

# Deuxièmement on décide manière un peu arbitraire du nombre de mots les plus fréquents à supprimer. On pourrait afficher un graphe d'évolution du nombre de mots pour se rendre compte et avoir une meilleure heuristique. 
most_freq = zip(*freq2.most_common(100))[0]

# On créé notre set de stopwords final qui cumule ainsi les 100 mots les plus fréquents du corpus ainsi que l'ensemble de stopwords par défaut présent dans la librairie NLTK
sw = set()
sw.update(stopwords)
sw.update(tuple(nltk.corpus.stopwords.words('french')))

NameError: ignored